In [2]:
import numpy as np
import requests
import wikipedia
import gensim
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import wikisearch
import nltk
from nltk.corpus import stopwords
import nltk.collocations
from nltk import FreqDist, word_tokenize
import string
import re

In [16]:
S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

TITLE = 'Jurassic Park (novel)'

PARAMS = {
    'action': "query",
    'titles': TITLE,
    'prop': "links",
    'pllimit': "max",
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
data = R.json()
#print(data['query']['title'])

In [29]:
data['query']['pages'][list(data['query']['pages'])[0]]['links'][:20]

[{'ns': 0, 'title': '2000 AD (comics)'},
 {'ns': 0, 'title': 'A Case of Need'},
 {'ns': 0, 'title': 'Airframe (novel)'},
 {'ns': 0, 'title': 'Alfred A. Knopf'},
 {'ns': 0, 'title': 'Amazon (video game)'},
 {'ns': 0, 'title': 'Amber'},
 {'ns': 0, 'title': 'Amphibian'},
 {'ns': 0, 'title': 'Amusement park'},
 {'ns': 0, 'title': 'Ancient DNA'},
 {'ns': 0, 'title': 'Andrew Ferguson'},
 {'ns': 0, 'title': 'Auxotrophy'},
 {'ns': 0, 'title': 'BILBY Award'},
 {'ns': 0, 'title': 'Backdoor (computing)'},
 {'ns': 0, 'title': 'Barnes & Noble'},
 {'ns': 0, 'title': 'Battle at Big Rock'},
 {'ns': 0, 'title': 'Beyond Westworld'},
 {'ns': 0, 'title': 'Binary (novel)'},
 {'ns': 0, 'title': 'Biotechnology'},
 {'ns': 0, 'title': 'Bird'},
 {'ns': 0, 'title': 'Canopy Flyer'}]

In [30]:
links = data['query']['pages'][list(data['query']['pages'])[0]]['links']

In [38]:
n = np.random.randint(0,len(links))
links[n]

{'ns': 0, 'title': 'DNA'}

In [46]:
n = np.random.randint(0,len(links))
NEXT = links[n]['title']

PARAMS = {
    'action': "query",
    'titles': NEXT,
    'prop': "links",
    'pllimit': "max",
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
next_data = R.json()
print(links[n]['title'])
print(next_data['query']['pages'][list(next_data['query']['pages'])[0]]['title'])

Jurassic World: Original Motion Picture Soundtrack
Jurassic World: Original Motion Picture Soundtrack


In [48]:
def get_links(TITLE):
    PARAMS = {
        'action': "query",
        'titles': TITLE,
        'prop': "links",
        'pllimit': "max",
        'format': "json",
    }

    R = S.get(url=URL, params=PARAMS)
    return R.json()

In [52]:
start = input()
hops = input()

print("0 : " + start)
title = start
for i in range(int(hops)):
    response = get_links(title)
    links = response['query']['pages'][list(response['query']['pages'])[0]]['links']
    n = np.random.randint(0,len(links))
    title = links[n]['title']
    print(str(i+1) + " : " + title)

 Jurassic Park (novel)
 5


0 : Jurassic Park (novel)
1 : Dragon curve
2 : Blancmange curve
3 : Archimedes
4 : Menander
5 : Akrai


In [54]:
jp = wikipedia.page('Jurassic Park (novel)')

Determine threshold for topic

In [102]:
get_50_most_common(wikipedia.page("Batman: No Man's Land"))[:10]

[('gotham', 39),
 ('city', 31),
 ('batman', 27),
 ('land', 22),
 ("man's", 20),
 ('gordon', 17),
 ('story', 11),
 ('two', 11),
 ('comics', 10),
 ('joker', 9)]

In [76]:
get_50_most_common(wikipedia.page('Catwoman'))[:10]

[('batman', 179),
 ('catwoman', 155),
 ('selina', 142),
 ('bruce', 50),
 ('two', 32),
 ('vol', 30),
 ('series', 29),
 ('story', 29),
 ('kyle', 28),
 ('one', 28)]

In [79]:
get_50_most_common(wikipedia.page('Bread'))[:20]

[('bread', 100),
 ('dough', 39),
 ('flour', 30),
 ('yeast', 25),
 ('leavening', 20),
 ('water', 18),
 ('breads', 18),
 ('used', 17),
 ('gluten', 14),
 ('wheat', 13),
 ('baking', 12),
 ('time', 12),
 ('made', 11),
 ('may', 11),
 ('protein', 11),
 ('process', 10),
 ('sourdough', 9),
 ('use', 9),
 ('also', 9),
 ('baked', 9)]

In [80]:
get_50_most_common(wikipedia.page('Cake'))[:10]

[('cake', 75),
 ('cakes', 60),
 ('flour', 19),
 ('made', 14),
 ('sugar', 13),
 ('butter', 13),
 ('baking', 13),
 ('bread', 12),
 ('sponge', 12),
 ('baked', 11)]

In [101]:
normalized_top_50(wikipedia.page("Batman: No Man's Land"))[:20]

[('gotham', '0.028'),
 ('city', '0.02225'),
 ('batman', '0.01938'),
 ('land', '0.01579'),
 ("man's", '0.01436'),
 ('gordon', '0.0122'),
 ('story', '0.007897'),
 ('two', '0.007897'),
 ('comics', '0.007179'),
 ('joker', '0.006461'),
 ('storyline', '0.005743'),
 ('gang', '0.005743'),
 ('police', '0.005743'),
 ('also', '0.005025'),
 ('territory', '0.005025'),
 ('dc', '0.004307'),
 ('issues', '0.004307'),
 ('new', '0.004307'),
 ('face', '0.004307'),
 ('luthor', '0.004307')]

In [115]:
l = wikipedia.page('Batman').links

In [123]:
l[:5]

['"Holy..."',
 '/Film',
 '100 (DC Comics)',
 '1939 in comics',
 '711 (Quality Comics)']

In [124]:
start = input()
target = input()

skiplist = ['Wikipedia', 'Category']

print("0 : " + start)
title = start
exit = False
for i in range(1000):
    try:
        page = wikipedia.page(title)
        top_20 = get_50_most_common(page)[:20]
        for word in top_20:
            if word[0] == target:
                exit = True
        if exit:
            break
        #response = get_links(title)
        links = page.links#response['query']['pages'][list(response['query']['pages'])[0]]['links']
        search_success = True
    except wikipedia.exceptions.DisambiguationError:
        search_success = False
    except wikipedia.exceptions.PageError:
        search_success = False
        
    n = np.random.randint(0,len(links))
    title = links[n]
    while any(sub in title for sub in skiplist):
        print(title)
        n = np.random.randint(0,len(links))
        title = links[n]
    if i % 100 == 0:
        print(".",end="")
print(str(i+1) + " Hops, Finish at : " + title)

 Superman
 Batman


0 : Superman
........

/Users/alexanderbailey/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/alexanderbailey/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html,features="lxml").find_all('li')


..1000 Hops, Finish at : Fundoshi


### Analyzing with Word2Vec

In [125]:

page = wikipedia.page('Catwoman')
page_text = page.content.split(".")

text = []

for clue in page_text:
    sentence = clue.translate(str.maketrans('','',string.punctuation)).split(' ')
    new_sent = [word.lower() for word in sentence]   
    text.append(new_sent)

In [88]:
text[:2]

[['twoface',
  'harvey',
  'dent',
  'is',
  'a',
  'fictional',
  'character',
  'appearing',
  'in',
  'comic',
  'books',
  'published',
  'by',
  'dc',
  'comics',
  'commonly',
  'as',
  'an',
  'adversary',
  'of',
  'the',
  'superhero',
  'batman'],
 ['',
  'the',
  'character',
  'was',
  'created',
  'by',
  'bob',
  'kane',
  '',
  'and',
  'bill',
  'finger',
  'and',
  'first',
  'appeared',
  'in',
  'detective',
  'comics',
  '66',
  'august',
  '1942']]

In [87]:
model = gensim.models.Word2Vec(text,sg=1)

In [89]:
model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

(26991, 68340)

In [90]:
model.corpus_total_words

13668

In [91]:
model.wv.most_similar("batman")

[('\ntwoface', 0.9981786012649536),
 ('video', 0.997878909111023),
 ('animated', 0.9978392720222473),
 ('universe', 0.9977647066116333),
 ('television', 0.9977205991744995),
 ('movie', 0.9977039098739624),
 ('film', 0.9976729154586792),
 ('liveaction', 0.9976227879524231),
 ('series', 0.997494637966156),
 ('first', 0.9974403977394104)]

In [3]:
cw_page = wikipedia.page('Catwoman')
cw_links = cw_page.links
cw_links[:10]

['"Holy..."',
 '711 (Quality Comics)',
 'A Death Worse Than Fate',
 'Abin Sur',
 'Ace Magazines (comics)',
 'Ace the Bat-Hound',
 'Action Comics',
 'African-American',
 'Air Wave',
 'Airboy']

In [4]:
words_stopped = [wikisearch.preprocess(link) for link in cw_links]
words_stopped[0][:10]

['holi']

In [5]:
dictionary = gensim.corpora.Dictionary(words_stopped)
bow_corpus = [dictionary.doc2bow(doc) for doc in words_stopped]

In [6]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [7]:
lda = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)

In [8]:
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.185*"comic" + 0.018*"marvel" + 0.018*"public" + 0.016*"black" + 0.011*"book" + 0.008*"charact" + 0.007*"captain" + 0.007*"gordon" + 0.007*"archi" + 0.006*"mask"
Topic: 1 
Words: 0.062*"comic" + 0.024*"batman" + 0.014*"superman" + 0.012*"dark" + 0.011*"book" + 0.011*"knight" + 0.010*"citi" + 0.009*"famili" + 0.009*"gotham" + 0.009*"ladi"
Topic: 2 
Words: 0.067*"batman" + 0.046*"comic" + 0.015*"robin" + 0.012*"list" + 0.010*"doctor" + 0.009*"charact" + 0.009*"phantom" + 0.008*"marvel" + 0.007*"penguin" + 0.007*"girl"
Topic: 3 
Words: 0.085*"comic" + 0.010*"wonder" + 0.010*"miss" + 0.009*"doctor" + 0.009*"atom" + 0.008*"american" + 0.008*"flash" + 0.007*"woman" + 0.006*"magazin" + 0.006*"aveng"
Topic: 4 
Words: 0.031*"charact" + 0.023*"leagu" + 0.022*"justic" + 0.013*"film" + 0.013*"green" + 0.013*"captain" + 0.009*"batgirl" + 0.009*"catwoman" + 0.008*"gotham" + 0.008*"lantern"


In [38]:
test_page = wikipedia.page('Baking')
test_words = wikisearch.preprocess(test_page)
tdictionary = gensim.corpora.Dictionary([test_words])
bow_test = [dictionary.doc2bow(test_words)]
test = lda[bow_test]

In [33]:
type(test)

gensim.interfaces.TransformedCorpus

In [39]:
for topic in test:
    print(topic)

[(0, 1.0)]
